# **Chapter 04-a. Detect Segment**

#### Object Detection (物件偵測)
* 影像分類問題中, 位置資訊不太重要, 只要物件有出現就好; 而物件偵測中, 位置資訊變得十分重要, 所以需要添加一些可以萃取位置資訊的元素
* 例如: 在 `convolution 的尾端`添加能夠偵測物件 bounding box 的元素, 就是 `YOLO (You Only Look Once)` 法
* 或是: 資訊在 convolution 骨幹的中間, 就可以建構`特徵金字塔(feature pyramid network, FPN)`

---
## **YOLO (You Only Look Once)**
* 最簡單、預測速度最快的架構, 常用於即時系統; 但準確率不一定是最高
* 架構上可以使用 Chapter 03 提到的任意一種模型, 只要把 prediction head 替換成 `oject detection and classification head (物件偵測與分類頭)` 即可
* YOLO 有非常多種變體, 這裡先介紹 YOLOv1

<img src="Images/YOLOv1.png" width=700/>

#### Object Detection Head
* `Grid(網格)`: YOLO 首先將圖片切割為 N×M 的 grid, 然後開始對於每個 grid 預測以它為中心可以抓出的物件(預測出來的 bounding bax 可以大於 grid 的大小, 但bounding box 的中心必須在該 grid 中, 且每個 grid 可以預測不只一種物件)
* 預測出來的結果有六項資訊: 中心的x,y座標 + width + height + confidence + predicted class
* xy值會經過tanh後落在[-1, 1]之間; width & height 會經過softmax後落在[0, 1]之間, 是相對於整張影像大小的比例; 而 confidence 也同樣介於[0, 1]間
* 原作者推薦在最後一層的特徵圖後, flatten 完再加上 fully-connected layer 以及適當的 activation function, 可以進一步增加準確性, 而 fully-connected layer 的輸出數量會是 N × M × (5 + num_of_class)

#### Loss Function
* Object detection 訓練資料的 label 會是 (ground truth bounding box + class), 因此 error 必須考慮到這兩項的錯誤情況
* 但是在同一個 grid 中心的預測框可能有不只一個. 所以挑選方式會是`取 網格內所有ground truth框 與 所有預測框 的 IOU(intersection over union, 交聯比)值的最高者` (註: IOU = 交集的面積/聯集的面積)

* `Object Presence Loss(物件存在損失)`: $L_{obj} = (1-confidence)^2$, 每個具有 ground truth bounding box 的 grid 都會計算
* `Object Absence Loss(物件缺失損失)`: $L_{noobj} = confidence^2$, 每個不具有 ground truth bounding box 的 grid 都會計算
* `Object Classification Loss(物件分類損失)`: $L_{class} = crossEntropy(p, \hat{p})$, $p$ 是目標類別的 one-hot vector, 而 $\hat{p}$ 是實際預測出的類別機率向量, 每個具有 ground truth bounding box 的 grid 都會計算
* `Bounding Box Loss(定界框損失)`: $L_{box} = (x-\hat{x})^2 + (y-\hat{y})^2 + (\sqrt{width}-{\sqrt{\hat{width}}})^2 + (\sqrt{height}-{\sqrt{\hat{height}}})^2$
* 因為大量的沒有物件grid所產生之小loss, 會壓過既有物件的單一grid之損失, 所以會對不同的loss項目給予不同之權重, 原作者推薦 $\lambda_{obj} = 1$, $\lambda_{noobj} = 0.5$, $\lambda_{class} = 1$, $\lambda_{box} = 5$

#### YOLO 之限制
* `YOLO 會預測每個 grid 只有一個類別`, 而`如果同一個grid中有許多不同類別的物件時, 效果就會不佳`
* 除非根據資料仔細調整 grid 大小, 否則 `YOLO 通常不能很好地處理小物件的集合`, 例如一群鳥
* `YOLO 定位物件的精確度通常較低`, 因為它位於 conv layer 的最後面, 所`接收到的 feature map 其實已經沒有太多空間資訊了`

---
## **⭐️ RetinaNet**
* 目前是單次偵測器(single-shot detector)的最先進技術之一

#### SSD(Single-Shot Detection)
* CNN 在處理影像時, `初始的 conv layer 會抓出邊緣和紋理等低階的細節`, 後面的層再將它們組合成愈來愈多具有語意價值的特徵, 但同時 pooling layer 會降低 feature map 的空間解析度
* 所以, 如果想要在每個階段都添加 detection head, 會變成初始層可以很好的定位物件, 但進行分類會不太準確, 因為高階的語意特徵需要到後層才會出現 -> `SSD(Single-Shot Detection)` 便是使用多個 detection head 連接到多個 feature map 進行預測, 這些特徵圖都位於卷積的末端幾層

#### **Feature Pyramid Network (FPN, 特徵金字塔網路)**
* `Bottom-Up 路徑`: 逐漸`精練 feature map 的語意資訊`, 並縮小空間維度
* `Top-Down 路徑`: 擁有較佳語意特徵的 feature map 先被 `upsample`(2x up, 為了讓圖片尺寸相同), 然後經過 `3×3 filter`(為了讓 upsample 的結果(artifact)更平滑一點), 再`逐元素加成`堆疊到更高的特徵圖中
  * 這裡的 upsample 採用最簡單的 nearest neighbor upsampling, 就是放大 N 倍會讓原始的每個 pixel 值變成新圖的 N×N 個 pixel 的值
* `Lateral 連接`: 使用 `1×1 的卷積讓所有特徵圖的 channel 數相等`, 以利於之後的逐元素加法
* 通常在 FPN 裡不使用 activation function
* 右圖中, **b** 即是 YOLO 的架構 (偵測頭在最後一層); **c** 是 SSD (偵測頭在末端幾層); **d** 是 FPN (從末端偵測並疊回)

<p float="left">
  <img src="Images/FPN.png" width="800" />
  <img src="Images/YOLO_vs_SSD_vs_FPN.png" width="300" /> 
</p>

* 通常 FPN 會使用 4~6 個從模型中間層提取出的 feature map 來連接 detection head
* 也很常使用 pre-trained 的 ResNet 與 EfficientNet 作為模型架構, 然後在他們後面加上額外的 pooling layer 或 conv layer, 可以提高對大型物件的偵測, 且因為他們遇到的輸入已經很小, 所以處理起來很快
* 連接到 FPN 的點建議選擇連續多個相似層、在進到 pooling layer 把解析度減半前的最後那層, 通常他會包含最強的特徵資訊

#### Anchor Box (錨框)
* `Anchor Box（錨框）` 是一組預先定義好的、不同尺寸與長寬比例的矩形框，固定貼在特徵圖（feature map）上的每個位置。
* 它們代表「預測的候選物件範圍」，模型會學習這些 anchor 是否包含物體、以及如何調整它們更接近真實物體（bounding box）。
* 每個位置會對應多個 anchor（例如不同尺度、比例），如 RetinaNet 使用 3 種長寬比 × 3 種縮放 = 9 個 anchor。
* 圖中會均勻被標注出一個個 anchor 中心, 每一個 anchor 中心都會有所有長寬比跟所有縮放大小的 anchor 疊在一起, 最後會計算出所有 anchor box 與 ground truth bounding box 的 IOU, anchor 會被分配給 IOU 最大的那個 anchor box; 但是若所有的 IOU 都不足 0.4, 則會被視為背景(偵測不到東西)

<img src="Images/Anchor_Box.png" width="350"/>

### **RetinaNet 原理**
* 假設共 K 種物件類別, 而 anchor box 有 B 種長寬比與縮放的組合
* 則 class prediction head 會預測出 B×K 個機率, 同時 object detection head 預測出 B 組 $\Delta{x}$, $\Delta{y}$, $\Delta{w}$ 與 $\Delta{h}$
* `p.142 ~ p.156 有非常詳盡（且難懂）的解釋, 有空記得回去看！`

##### 來看看 ChatGPT 怎麼說
* 2017 年由 Facebook AI 提出，是一種 單階段（one-stage）目標檢測器。
* 問題背景：傳統 one-stage 檢測在準確度上往往輸給 two-stage 檢測（如 Faster R-CNN），原因是`正負樣本數嚴重不平衡`（大部分 anchor 是背景）。
* 解法：引入 `Focal Loss`，減少模型在大量易分類負樣本上的損失權重，讓訓練更專注於困難樣本。
<br /><br />
* 架構組成
    1. Backbone：通常用 ResNet + FPN, 提取多尺度特徵圖（P3 ~ P7）。
    2. Classification Subnet：對每個 anchor 預測物件類別機率（多分類 + 背景）。
    3. Regression Subnet：預測 anchor 的位置偏移量（bounding box regression）。
    4. Loss Function：分類使用 Focal Loss（降低簡單負樣本的權重），回歸使用 Smooth L1 Loss
<br /><br />
* 優缺點
    * ✅ 精度接近 two-stage（Faster R-CNN），速度保留 one-stage 優勢
    * ✅ 適合多尺度檢測（FPN 幫助大）
    * ❌ 訓練需要調參（focal loss 的 α、γ）
    * ❌ 對超小物體的檢測仍不如一些 two-stage 方法